In [27]:
# data
import numpy as np

np.random.seed(0)

ndims = 1
ndata = 100
X = np.random.randn(ndata, ndims)
w_ = np.random.randn(ndims)  # hidden
noise_ = 0.1 * np.random.randn(ndata)  # hidden

y_obs = X.dot(w_) + noise_

In [28]:
import tensorflow as tf
import tf_keras

import tensorflow_probability as tfp
tfd = tfp.distributions

In [29]:
# Credit to https://colcarroll.github.io/ppl-api/ for this example

X_tensor = tf.convert_to_tensor(X, dtype='float32')

@tf.function
def target_log_prob_fn(w):
    w_dist = tfd.Normal(loc=tf.zeros((ndims, 1)), scale=1.0, name="w")
    w_prob = tf.reduce_sum(w_dist.log_prob(w))
    y_dist = tfd.Normal(loc=tf.matmul(X_tensor, w), scale=0.1, name="y")
    y_prob = tf.reduce_sum(y_dist.log_prob(y_obs.reshape(-1, 1)))
    return w_prob + y_prob


# Initialize the HMC transition kernel.
num_results = 1000
num_burnin_steps = 500
adaptive_hmc = tfp.mcmc.SimpleStepSizeAdaptation(
    tfp.mcmc.HamiltonianMonteCarlo(
        target_log_prob_fn=target_log_prob_fn,
        num_leapfrog_steps=4,
        step_size=0.01),
    num_adaptation_steps=int(num_burnin_steps * 0.8))

samples, is_accepted = tfp.mcmc.sample_chain(
    num_results=num_results,
    num_burnin_steps=num_burnin_steps,
    current_state=tf.zeros((ndims, 1)),
    kernel=adaptive_hmc,
    trace_fn=lambda _, pkr: pkr.inner_results.is_accepted)

In [31]:
# 1000 samples, single parameter
samples.shape

TensorShape([1000, 1, 1])

In [42]:
posterior = {"b0": samples.numpy()[:,0,0]}

In [44]:
az.summary(posterior)

arviz - WARNING - Shape validation failed: input_shape: (1, 1000), minimum_shape: (chains=2, draws=4)


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
b0,1.892,0.011,1.875,1.914,0.0,0.0,479.0,340.0,NaN


Another good reference is: https://www.tensorflow.org/probability/examples/Modeling_with_JointDistribution

In [12]:
import arviz as az

tfd = tfp.distributions
tfb = tfp.bijectors

dtype = tf.float64

In [15]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

dfhogg = pd.DataFrame(np.array([[1, 201, 592, 61, 9, -0.84],
                                 [2, 244, 401, 25, 4, 0.31],
                                 [3, 47, 583, 38, 11, 0.64],
                                 [4, 287, 402, 15, 7, -0.27],
                                 [5, 203, 495, 21, 5, -0.33],
                                 [6, 58, 173, 15, 9, 0.67],
                                 [7, 210, 479, 27, 4, -0.02],
                                 [8, 202, 504, 14, 4, -0.05],
                                 [9, 198, 510, 30, 11, -0.84],
                                 [10, 158, 416, 16, 7, -0.69],
                                 [11, 165, 393, 14, 5, 0.30],
                                 [12, 201, 442, 25, 5, -0.46],
                                 [13, 157, 317, 52, 5, -0.03],
                                 [14, 131, 311, 16, 6, 0.50],
                                 [15, 166, 400, 34, 6, 0.73],
                                 [16, 160, 337, 31, 5, -0.52],
                                 [17, 186, 423, 42, 9, 0.90],
                                 [18, 125, 334, 26, 8, 0.40],
                                 [19, 218, 533, 16, 6, -0.78],
                                 [20, 146, 344, 22, 5, -0.56]]),
                   columns=['id','x','y','sigma_y','sigma_x','rho_xy'])


## for convenience zero-base the 'id' and use as index
dfhogg['id'] = dfhogg['id'] - 1
dfhogg.set_index('id', inplace=True)

## standardize (mean center and divide by 1 sd)
dfhoggs = (dfhogg[['x','y']] - dfhogg[['x','y']].mean(0)) / dfhogg[['x','y']].std(0)
dfhoggs['sigma_y'] = dfhogg['sigma_y'] / dfhogg['y'].std(0)
dfhoggs['sigma_x'] = dfhogg['sigma_x'] / dfhogg['x'].std(0)

# def plot_hoggs(dfhoggs):
#   ## create xlims ylims for plotting
#   xlims = (dfhoggs['x'].min() - np.ptp(dfhoggs['x'])/5,
#            dfhoggs['x'].max() + np.ptp(dfhoggs['x'])/5)
#   ylims = (dfhoggs['y'].min() - np.ptp(dfhoggs['y'])/5,
#            dfhoggs['y'].max() + np.ptp(dfhoggs['y'])/5)

#   ## scatterplot the standardized data
#   g = sns.FacetGrid(dfhoggs, size=8)
#   _ = g.map(plt.errorbar, 'x', 'y', 'sigma_y', 'sigma_x', marker="o", ls='')
#   _ = g.axes[0][0].set_ylim(ylims)
#   _ = g.axes[0][0].set_xlim(xlims)

#   plt.subplots_adjust(top=0.92)
#   _ = g.figure.suptitle('Scatterplot of Hogg 2010 dataset after standardization', fontsize=16)
#   return g, xlims, ylims

# g = plot_hoggs(dfhoggs)

In [17]:
X_np = dfhoggs['x'].values
sigma_y_np = dfhoggs['sigma_y'].values
Y_np = dfhoggs['y'].values

In [20]:
# mdl_ols = tfd.JointDistributionSequential([
#     # b0 ~ Normal(0, 1)
#     tfd.Normal(loc=tf.cast(0, dtype), scale=1.),
#     # b1 ~ Normal(0, 1)
#     tfd.Normal(loc=tf.cast(0, dtype), scale=1.),
#     # x ~ Normal(b0+b1*X, 1)
#     lambda b1, b0: tfd.Normal(
#       # Parameter transformation
#       loc=b0 + b1*X_np,
#       scale=sigma_y_np)
# ])

mdl_ols_ = tfd.JointDistributionSequential([
    # b0
    tfd.Normal(loc=tf.cast(0, dtype), scale=1.),
    # b1
    tfd.Normal(loc=tf.cast(0, dtype), scale=1.),
    # likelihood
    #   Using Independent to ensure the log_prob is not incorrectly broadcasted
    lambda b1, b0: tfd.Independent(
        tfd.Normal(
            # Parameter transformation
            # b1 shape: (batch_shape), X shape (num_obs): we want result to have
            # shape (batch_shape, num_obs)
            loc=b0 + b1*X_np,
            scale=sigma_y_np),
        reinterpreted_batch_ndims=1
    ),
])

In [21]:
mdl_ols.resolve_graph()

(('b0', ()), ('b1', ()), ('x', ('b1', 'b0')))

In [22]:
print(mdl_ols_.sample_distributions()[0][-1])
print(mdl_ols.sample_distributions()[0][-1])

tfp.distributions.Independent("IndependentNormal", batch_shape=[], event_shape=[20], dtype=float64)
tfp.distributions.Normal("Normal", batch_shape=[20], event_shape=[], dtype=float64)


In [23]:
Root = tfd.JointDistributionCoroutine.Root  # Convenient alias.
def model():
  b1 = yield Root(tfd.Normal(loc=tf.cast(0, dtype), scale=1.))
  b0 = yield Root(tfd.Normal(loc=tf.cast(0, dtype), scale=1.))
  yhat = b0 + b1*X_np
  likelihood = yield tfd.Independent(
        tfd.Normal(loc=yhat, scale=sigma_y_np),
        reinterpreted_batch_ndims=1
    )

mdl_ols_coroutine = tfd.JointDistributionCoroutine(model)
mdl_ols_coroutine.log_prob(mdl_ols_coroutine.sample())

<tf.Tensor: shape=(), dtype=float64, numpy=-0.06748552886278159>

In [24]:
mdl_ols_coroutine.sample()  # output is a tuple

StructTuple(
  var0=<tf.Tensor: shape=(), dtype=float64, numpy=0.8899468403586868>,
  var1=<tf.Tensor: shape=(), dtype=float64, numpy=0.7728553511496148>,
  var2=<tf.Tensor: shape=(20,), dtype=float64, numpy=
    array([ 1.84876111,  1.64697786, -1.46597035,  2.52133377,  1.18839469,
           -1.07816046,  0.8359043 ,  1.05324044,  1.55456861,  0.6797094 ,
            0.5811698 ,  1.51899324, -0.39635904,  0.12090469,  0.3083027 ,
            0.96955596,  0.53666496, -0.17332018,  1.37816081,  0.38455846])>
)